In [51]:
import pandas as pd
from pyproj import Proj
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn import cross_validation
from sklearn.metrics import mean_squared_error
import seaborn as sns
sns.set(color_codes=True)
import collections

%matplotlib inline

In [84]:
data_path = "data/merged/manhattan_brooklyn_statenisland_bronx_queens_2003_2016.csv"
df = pd.read_csv(data_path, low_memory = True)
df.head()

,zipcode,ltdheight,splitzone,easements,comarea,resarea,numbldgs,numfloors,unitsres,unitstotal,...,proxcode_2.0,lottype_0.0,lottype_1.0,lottype_2.0,lottype_3.0,lottype_4.0,lottype_5.0,lottype_6.0,tax_class_at_time_of_sale_1,tax_class_at_time_of_sale_2
0,10004.0,0.0,0.0,0.0,1888126.0,0.0,1.0,50.0,0.0,52.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10004.0,0.0,0.0,0.0,1888126.0,0.0,1.0,50.0,0.0,52.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10004.0,0.0,0.0,0.0,1888126.0,0.0,1.0,50.0,0.0,52.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10004.0,0.0,0.0,0.0,1888126.0,0.0,1.0,50.0,0.0,52.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004.0,0.0,0.0,0.0,1888126.0,0.0,1.0,50.0,0.0,52.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
df.shape

(2108308, 180)

In [87]:
df[['gross_sqft_pluto','sale_price','price_per_sqft']].head()

,gross_sqft_pluto,sale_price,price_per_sqft
0,1888126.0,0,0.000000e+00
1,1888126.0,0,0.000000e+00
2,1888126.0,0,0.000000e+00
3,1888126.0,0,0.000000e+00
4,1888126.0,1,5.296257e-07


In [88]:
df = df[df['price_per_sqft'] > 5]
df.shape

(835376, 180)

In [89]:
df.columns.values

array(['zipcode', 'ltdheight', 'splitzone', 'easements', 'comarea',
       'resarea', 'numbldgs', 'numfloors', 'unitsres', 'unitstotal',
       'lotfront', 'lotdepth', 'bldgfront', 'bldgdepth', 'irrlotcode',
       'bsmtcode', 'yearbuilt', 'builtcode', 'histdist', 'landmark',
       'condono', 'xcoord', 'ycoord', 'zonemap', 'latitude', 'longitude',
       'gross_sqft_pluto', 'garage', 'extension', 'countalter',
       'sale_price', 'sale_date', 'year_built', 'residential_units',
       'commercial_units', 'total_units', 'price_per_sqft',
       'schooldist_mv', 'council_mv', 'zipcode_mv', 'ownertype_mv',
       'numbldgs_mv', 'unitsres_mv', 'unitstotal_mv', 'lotfront_mv',
       'lotdepth_mv', 'bldgfront_mv', 'bldgdepth_mv', 'proxcode_mv',
       'yearbuilt_mv', 'xcoord_mv', 'ycoord_mv', 'zonemap_mv',
       'latitude_mv', 'longitude_mv', 'borough_BK', 'borough_BX',
       'borough_MN', 'borough_QN', 'schooldist_1.0', 'schooldist_2.0',
       'schooldist_3.0', 'schooldist_4.0', 'school

In [78]:
count = collections.Counter(df['price_per_sqft'].astype(int))
count

Counter({0: 593176,
         1: 110344,
         2: 108648,
         3: 82488,
         4: 69512,
         5: 54444,
         6: 45828,
         7: 41824,
         8: 34420,
         9: 31460,
         10: 26272,
         11: 24484,
         12: 21100,
         13: 19772,
         14: 18024,
         15: 16560,
         16: 16648,
         17: 14932,
         18: 13576,
         19: 12780,
         20: 11332,
         21: 10972,
         22: 9960,
         23: 9720,
         24: 9552,
         25: 8276,
         26: 7676,
         27: 7812,
         28: 7424,
         29: 6604,
         30: 6808,
         31: 6196,
         32: 5880,
         33: 5840,
         34: 5256,
         35: 4864,
         36: 4928,
         37: 4656,
         38: 4660,
         39: 3780,
         40: 4272,
         41: 3688,
         42: 4092,
         43: 3812,
         44: 3428,
         45: 3480,
         46: 3512,
         47: 2900,
         48: 2776,
         49: 2952,
         50: 3188,
         51: 277

In [80]:
outliers = df.loc[df['price_per_sqft'].astype(int).isin([1,2,3,4,5,6912,7655,7949,10315,12089,14568])]
outliers[['latitude','longitude','price_per_sqft','sale_price','year_built','sale_date']].to_csv("outliers.csv")

In [65]:
#output distribution of target variable to visualize in Tableau
import csv
x = df['price_per_sqft']
count = collections.Counter(x.astype(int))
with open("price_per_sqft_counts.csv",'w') as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(['Price Per Sqft', 'Frequency'])
    for key, count in count.items():
        writer.writerow([key, count])

In [68]:
#output distribution of sale price to visualize in Tableau
import csv
x = df['sale_price']
count = collections.Counter(x.astype(int))
with open("sale_price_counts.csv",'w') as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(['Sale Price', 'Frequency'])
    for key, count in count.items():
        writer.writerow([key, count])

In [90]:
def drop_cols(data, cols):
    return data.drop(cols, axis = 1)

In [91]:
df = drop_cols(df, ['zonemap','sale_date','sale_price'])

In [92]:
def split_data(data):
    '''
    Splits data into training and test sets (0.8/0.2)
        Args: 
            data: Pandas dataframe
        Returns:
            data_train: Pandas dataframe used for training
            data_test: Pandas dataframe used for testing
    
    '''
    #Convert 'int64' into float; otherwise, sklearn throws a warning message
    columns = data.columns.values
    non_float = []
    for col in columns:
        if data[col].dtype != np.float64:
            non_float.append(col)
        for col in non_float:
            data[col] = data[col].astype(float)
    #drop NaN for crucial columns
    data= data.dropna(how = 'any', subset = ['latitude','longitude','price_per_sqft'])   
    #Split the data
    split = cross_validation.ShuffleSplit(data.shape[0], n_iter=1, train_size = 0.7, test_size=.3, random_state = 1)

    for train, test in split:
        train_index = train
        test_index = test
    data_train = data.ix[train_index,:]
    data_test = data.ix[test_index,:]
    data_train.reset_index(drop=True, inplace=True)
    data_test.reset_index(drop=True, inplace=True)
    return data_train, data_test

In [93]:
data_train, data_test = split_data(df)

In [94]:
def fill_na(data_train, data_test):
    '''
    Fills NaN values with the mean of the column. Note we have already created dummy variables
    for columns with missing values.
    
    Args:
        data_train: Pandas dataframe used for training.
        data_test: Pandas dataframe used for testing.
    Returns:
        data_train: Pandas dataframe with no NaN values, ready for modeling.
        data_test: Pandas dataframe with no NaN values, ready for testing.
    
    '''
    data_train = data_train.apply(lambda x: x.fillna(x.mean()),axis=0)
    data_test = data_test.apply(lambda x: x.fillna(x.mean()),axis=0)
    return data_train, data_test

In [95]:
data_train, data_test = fill_na(data_train, data_test)

In [96]:
print(data_train.shape, data_test.shape)


(583839, 177) (250217, 177)


In [98]:
cols = list(data_train.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('price_per_sqft')) #Remove b from list
data_train = data_train[cols+['price_per_sqft']]
data_test = data_test[cols+['price_per_sqft']]
data_train.columns.values

array(['zipcode', 'ltdheight', 'splitzone', 'easements', 'comarea',
       'resarea', 'numbldgs', 'numfloors', 'unitsres', 'unitstotal',
       'lotfront', 'lotdepth', 'bldgfront', 'bldgdepth', 'irrlotcode',
       'bsmtcode', 'yearbuilt', 'builtcode', 'histdist', 'landmark',
       'condono', 'xcoord', 'ycoord', 'latitude', 'longitude',
       'gross_sqft_pluto', 'garage', 'extension', 'countalter',
       'year_built', 'residential_units', 'commercial_units',
       'total_units', 'schooldist_mv', 'council_mv', 'zipcode_mv',
       'ownertype_mv', 'numbldgs_mv', 'unitsres_mv', 'unitstotal_mv',
       'lotfront_mv', 'lotdepth_mv', 'bldgfront_mv', 'bldgdepth_mv',
       'proxcode_mv', 'yearbuilt_mv', 'xcoord_mv', 'ycoord_mv',
       'zonemap_mv', 'latitude_mv', 'longitude_mv', 'borough_BK',
       'borough_BX', 'borough_MN', 'borough_QN', 'schooldist_1.0',
       'schooldist_2.0', 'schooldist_3.0', 'schooldist_4.0',
       'schooldist_5.0', 'schooldist_6.0', 'schooldist_7.0',
       's

In [99]:
X_train = data_train.ix[:,:-1]
y_train = data_train.ix[:,-1]
X_test = data_test.ix[:,:-1]
y_test = data_test.ix[:,-1]
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
mse = mean_squared_error(y_test, regr.predict(X_test))
print('Mean_squared_error', mse)

Mean_squared_error 3835.70896625


In [100]:
from sklearn.ensemble import RandomForestRegressor

RF_reg_final = RandomForestRegressor(n_estimators=100, n_jobs = -1)
RF_reg_final.fit(X_train, y_train)
print(mean_squared_error(y_test, RF_reg_final.predict(X_test)))

730.063776792


In [101]:
feature_importance =  RF_reg_final.feature_importances_
indices = np.argsort(feature_importance)[::-1][:27]

feature_dct = {}
# Print the feature ranking
print("Feature ranking:")

for f in range(27):
    feature_dct[X_test.columns.values[indices][f]] = feature_importance[indices[f]]
feature_dct

Feature ranking:


{'bldgclass_RH': 0.0081431358501530132,
 'bldgdepth': 0.0074211088957864694,
 'bldgfront': 0.019628587386128847,
 'comarea': 0.011652903133376637,
 'commercial_units': 0.031946459639348723,
 'condono': 0.028151564113191516,
 'council_4.0': 0.064738883533382613,
 'gross_sqft_pluto': 0.27705120233849928,
 'latitude': 0.019486311360964726,
 'longitude': 0.0084538998537445446,
 'lotdepth': 0.0067174627188642602,
 'lotfront': 0.018881395106410032,
 'lottype_0.0': 0.0027191197417211028,
 'numfloors': 0.01730054916230649,
 'resarea': 0.016389573559105912,
 'residential_units': 0.014207518233361621,
 'tax_class_at_time_of_sale_2': 0.010031960021858839,
 'total_units': 0.0039463066680926256,
 'unitsres': 0.0074706839333408236,
 'unitsres_mv': 0.010478671448495602,
 'unitstotal': 0.24622187605693124,
 'unitstotal_mv': 0.016018398134770021,
 'xcoord': 0.010404188978540976,
 'ycoord': 0.024354671803658513,
 'year_built': 0.049783134703004689,
 'yearbuilt': 0.038630468418809287,
 'zipcode': 0.00412

In [103]:
data_test['predicted'] = RF_reg_final.predict(X_test)
data_test['percent_difference'] = 100*(np.abs(data_test['predicted'] - data_test['price_per_sqft']).astype(float) / data_test['price_per_sqft'])
data_test[['price_per_sqft','predicted','percent_difference']].head()

,price_per_sqft,predicted,percent_difference
0,44.098298,44.479898,0.865338
1,31.059346,37.080267,19.385218
2,44.098298,44.479898,0.865338
3,44.098298,44.479898,0.865338
4,7.244468,7.732797,6.740715


In [105]:
acc = 100 * (data_test[data_test['percent_difference'] < 10.0].shape[0]/ data_test.shape[0])
acc

69.82179468221584

In [106]:
predicted = RF_reg_final.predict(X_test)
percent_diff = 100*(np.abs(predicted - y_test).astype(float) / y_test)

In [107]:
acc = 100 * (sum(i < 10. for i in percent_diff)/ len(percent_diff))
acc

69.821794682215838